短期分析BBC關於Brexit的新聞：2019/1/1 ~ 2019/4/30

In [65]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import ssl
import datetime
import requests

In [66]:
driver = webdriver.Chrome("/Library/Frameworks/Python.framework/Versions/3.6/chromedriver")
#driver.get("https://www.bbc.com/news/topics/cwlw3xz0lvvt/brexit")
driver.get("https://www.bbc.co.uk/search?q=Brexit")


more_path = '//*[@id = "search-content"]/nav[1]/a' # find click button path
context = ssl._create_unverified_context()


for i in range(35):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element_by_xpath(more_path).click()
    driver.implicitly_wait(5)
pageSource = driver.page_source
bsObj = BeautifulSoup(pageSource)

In [67]:
try:
    articles_v = bsObj.find_all("article", {"class" : "has_image media-video"})
    articles_t = bsObj.find_all("article", {"class" : "has_image media-text"})
except:
    pass


def process_data(obj):
    news_list = []
    for news in obj:
        try:
            date = news.find('time')['datetime'][:10]
            title = news.find("h1", {"itemprop" : "headline"}).get_text()
            summary = news.find("p", {"class" : "summary long"}).get_text()
            types = news.find('span')['data-site']
            links = news.find('a')['href']
            news_list.append({
                'date' : date,
                'title' : title,
                'summary' : summary,
                'type' : types,
                'link' : links})
        except:
            pass
    return news_list

videos = process_data(articles_v)
texts = process_data(articles_t)

if len(texts) != 0:
    for adict in texts:
        videos.append(adict)
print(len(videos))


290


In [68]:
import pandas as pd
pd.DataFrame(videos)[['date', 'title', 'type', 'summary', 'link']].head()

,date,title,type,summary,link
0,2019-04-09,Spotlight: Brexit Drama,programmes,… that will determine the fate of Brexit.\n\nP...,https://www.bbc.co.uk/programmes/m00043x9
1,2018-05-26,The Brexit Lectures: Series 1: Nick Clegg on B...,programmes,"… a lecture on the future of Brexit, from Mond...",https://www.bbc.co.uk/programmes/b0b4nz8k
2,2018-05-05,The Brexit Lectures: Series 1: Liam Fox on Brexit,programmes,"… of Brexit, from Monday 30 April.…",https://www.bbc.co.uk/programmes/b0b26br4
3,2019-04-30,Welsh Assembly: Brexit Statement,programmes,…Brexit Minister Jeremy Miles making a stateme...,https://www.bbc.co.uk/programmes/m000524m
4,2019-04-12,Brexit Party Campaign Launch,programmes,…Leader of the Brexit Party Nigel Farage MEP l...,https://www.bbc.co.uk/programmes/m0004mqj


In [69]:
# BBC新聞有分programmes 和 news，因為要進行文字分析，所以只取出類別為news的新聞
category = {'programmes': 0, 'news': 0}
for news in videos:
    if news['type'] == 'programmes':
        category['programmes'] += 1
    else:
        category['news'] += 1
print(category)

{'programmes': 174, 'news': 116}


In [76]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
proxies = ""

news_to_ana = []
for news in videos:
    if news['type'] == 'news':
        #print("Y")
        try:
            html = news['link']
            #print(html)
            re = requests.get(html, headers = headers, proxies = proxies)
            bs = BeautifulSoup(re.content, 'lxml')
            contents= []
            for p in bs.select('.story-body__inner p')[:]:
                contents.append(p.text.strip())
            article = ' '.join(contents)
            #print(article)
            if article != '':
                news_to_ana.append({
                'date': news['date'],
                'title' : news['title'],
                'content' : article})
        except:   
            pass
print(news_to_ana)
               
'''
html = 'https://www.bbc.com/news/uk-northern-ireland-47718367'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
proxies = ""

re = requests.get(html, headers = headers, proxies = proxies)
bs = BeautifulSoup(re.content, 'lxml')

contents= []
for p in bs.select('.story-body__inner p')[:]:
    contents.append(p.text.strip())
article = ' '.join(contents)
print(article)
'''

[{'date': '2019-05-05', 'title': 'Theresa May urges Jeremy Corbyn to do a Brexit deal', 'content': 'Theresa May has called for Jeremy Corbyn to "put their differences aside" and agree a Brexit deal. The UK was supposed to leave the EU on 29 March - but the deadline was delayed until 31 October, after MPs rejected Theresa May\'s withdrawal agreement three times. Mrs May is now seeking Labour support to get an agreement through Parliament. Writing in the Mail on Sunday, she said they should "listen to what voters said" in Thursday\'s local elections. The Conservatives lost 1,334 councillors, while Labour failed to make expected gains, instead losing 82 seats. The Liberal Democrats benefited from Tory losses, gaining 703 seats, with the Greens and independents also making gains. The prime minister blamed the Brexit impasse for the losses - but said the elections gave "fresh urgency" to find a way to "break the deadlock". Mrs May said she hopes to find a "unified, cross-party position" wit

'\nhtml = \'https://www.bbc.com/news/uk-northern-ireland-47718367\'\nheaders = {\'User-Agent\': \'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36\'}\nproxies = ""\n\nre = requests.get(html, headers = headers, proxies = proxies)\nbs = BeautifulSoup(re.content, \'lxml\')\n\ncontents= []\nfor p in bs.select(\'.story-body__inner p\')[:]:\n    contents.append(p.text.strip())\narticle = \' \'.join(contents)\nprint(article)\n'

In [75]:
pd.DataFrame(news_to_ana)[['date', 'title', 'content']].head()

,date,title,content
0,2019-05-03,Local elections: Tories 'accept' Brexit frustr...,
1,2019-05-04,Brexit: Professor walks 500 miles from Germany...,
2,2019-05-03,Jeremy Hunt: Changing leader would not help Br...,
3,2019-05-03,Labour MP Hilary Benn says Brexit looms over v...,
4,2019-05-03,Local elections: Brexit 'dissatisfaction hitti...,


In [72]:
import pickle

file = open('Brexit_BBC.pickle', 'wb')
pickle.dump(news_to_ana, file)
file.close()